In [1]:
import pandas as pd
import functions.common as cf
import functions.plotting as pf
import xarray as xr

**Define Datasets Review List**

In [2]:
reviewlist = pd.read_csv('define_datareview_list.csv')

In [3]:
col = list(reviewlist.columns)
refdes = reviewlist[col[1]][0].split('/')[-1].split('_')[1][0:27]
dr_data = cf.refdes_datareview_json(refdes)

In [4]:
df = pd.DataFrame()
for ii in range(len(reviewlist)):         
    deploy_num = int(reviewlist[col[0]][ii].split('t')[-1])
    deploy_info = cf.get_deployment_information(dr_data, deploy_num)
    deploy_depth = deploy_info['deployment_depth']
    
    # Check deployment pressure from asset management against pressure variable in file
    ds = xr.open_dataset(reviewlist[col[1]][ii], mask_and_scale=False)
    ds = ds.swap_dims({'obs': 'time'})
    press = pf.pressure_var(ds, list(ds.coords.keys()))
    if press is None:
        press = pf.pressure_var(ds, list(ds.data_vars.keys()))     
    pressure_compare, pressure_max, pressure_mean = cf.calculate_mean_pressure(press, ds, refdes, deploy_depth)
        
    df0 = pd.DataFrame({'method': reviewlist[col[2]][ii],
                        'deploy_depth': [deploy_depth],
                        'data_pressure_max': [pressure_max],
                        'pressure_compare': [pressure_compare]
                        }, index=[deploy_num])
    df = df.append(df0)

In [5]:
list(df.index)
depl_loc_diff = pd.DataFrame(columns=list(df.index), index=list(df.index))
depl_loc_diff

,1,2,3,4
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [6]:
y = cf.deploy_location_check(refdes)
delpoy_y_loc = pd.DataFrame()
for ii in y:
    delpoy_y_loc = delpoy_y_loc.append(pd.DataFrame({'loc-t0':ii.split('_')[4].split('D')[1],
                                                     'loc-tn':ii.split('_')[2].split('D')[1],
                                                     'diff (km)': round(y[ii])}, index=[ii]))
delpoy_y_loc = delpoy_y_loc.sort_values(by=['loc-t0'])
delpoy_y_loc.index = delpoy_y_loc['loc-t0']
delpoy_y_loc = delpoy_y_loc[['loc-tn','diff (km)']]


xy = delpoy_y_loc.groupby(delpoy_y_loc.index)

In [7]:
for jj in list(df.index): 
    tn = xy.get_group(str(jj))
    for ii in list(df.index):
        x = tn[tn['loc-tn'] == str(ii)]
        if len(x) != 0: 
            depl_loc_diff[ii][jj] = tn['diff (km)'][tn['loc-tn'] == str(ii)].values[0]
            depl_loc_diff[jj][ii] = tn['diff (km)'][tn['loc-tn'] == str(ii)].values[0]
            print(ii, jj, len(x),depl_loc_diff[ii][jj], depl_loc_diff[jj][ii])
        else:
            depl_loc_diff[ii][jj] = '-' 
test

2 1 1 0 0
3 1 1 18 18
4 1 1 28 28
3 2 1 18 18
4 2 1 28 28
4 3 1 28 28


NameError: name 'test' is not defined